##예제
1. drinks.csv 파일 로드
2. 결측치를 확인
3. 데이터프레임의 정보 출력
4. continent 컬럼에서 결측치 존재 -> 결측치 'OT'로 대체
5. continent 별 spirit_servings 컬럼의 평균,최소,최대,합계를 출력
6. total_litres_of_pure_alcohol 컬럼의 평균보다 높은 continent가 어디인지 출력
7. 국가별 술 소비량 대비 알콜 비율 컬럼을 추가

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
# 파일 로드
drinks = pd.read_csv("../csv/drinks.csv")

In [ ]:
# 결측치 확인
drinks.isna().sum()

In [ ]:
# 데이터프레임의 정보 확인
drinks.info()

In [ ]:
# 결측치를 'OT'로 대체
# 결측치를 대체하는 함수 -> fillna('OT')
# drinks.loc[결측치인 조건,'continent'] = 'OT'

drinks['continent'] = drinks['continent'].fillna('OT')
drinks.loc[(drinks['continent'].isna()),'continent'] = 'OT'

In [ ]:
# continent별 spirit_servings컬럼의 평균,최소,최대,합계를 출력
group_data = drinks[['continent','spirit_servings']].groupby('continent').agg(['mean','min','max','sum'])
# drinks.describe()

In [ ]:
pd.pivot_table(
    drinks,
    index='continent',
    values='spirit_servings',
    aggfunc=['mean','min','max','sum']

)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 바형그래프
# 한 화면에 4개의 그래프
# bar() x와 y의 값이 필수 항목
x = group_data.index

#subplot(행의수,열의수,영역지정)
plt.subplot(2,2,1)
plt.bar(x, group_data['spirit_servings']['mean'])
plt.subplot(2,2,2)
plt.bar(x, group_data['spirit_servings']['min'])
plt.subplot(2,2,3)
plt.bar(x, group_data['spirit_servings']['max'])
plt.subplot(2,2,4)
plt.bar(x, group_data['spirit_servings']['sum'])  #sum의 값이 다른값보다 크기 때문에 서브플롯으로 작성

plt.show()


In [ ]:
# 전체 알콜의 소비량의 평균보다 높은 대륙은 어디인가?

# 전체 알콜의 소비량의 평균
total_mean = drinks['total_litres_of_pure_alcohol'].mean()

In [ ]:
group_data2 = drinks[['continent','total_litres_of_pure_alcohol']].groupby('continent').mean()['total_litres_of_pure_alcohol']

In [ ]:
group_data2[group_data2 > total_mean]

In [ ]:
# 국가별 술 소비량 대비 알콜의 비율 생성
drinks

In [ ]:
# 국가별 술 소비량 파생변수 생성
drinks['total_servings'] = drinks['beer_servings'] + drinks['spirit_servings']+ drinks['wine_servings']

In [ ]:
drinks.head(2)

In [ ]:
# 술 소비량 대비 알콜 비율 파생변수 생성
# 알콜 소비량 / 술 소비량 * 100
# 결측치가 나온다면 0으로 대체

drinks['alcohol_rate'] = (drinks['total_litres_of_pure_alcohol']/drinks['total_servings']*100).fillna(0)

In [ ]:
# 가장 독한술을 먹는 나라는 어디인가?
# 우리나라는 얼마나 독한 술을 마시는가?
# 북한은 얼마나 독한 술을 먹는가?
drinks_rank = drinks.sort_values('alcohol_rate', ascending = False, ignore_index=True)

In [ ]:
drinks_rank.loc[drinks_rank['country'] == 'South Korea']

In [ ]:
drinks_rank.loc[drinks_rank['country'] == 'North Korea']

In [ ]:
# 시각화
# 우리나라를 그래프에서 표기
x = list(range(len(drinks_rank)))
y = drinks_rank['alcohol_rate']

# 국가 리스트
country_list = drinks_rank['country'].tolist()

bar_list = plt.bar(x,y)

korea_rank = country_list.index("South Korea")
korea_alcohol_rate = drinks_rank.loc[drinks_rank['country'] == 'South Korea','alcohol_rate'].values[0]

# 그래프 커스텀

# South Korea의 막대의 색상을 변경 , 여기서 index는 위치를 가리킨다
# bar_list[country_list.index('South Korea')].set_color('r')
bar_list[korea_rank].set_color("r")

# 그래프에 글씨와 화살표 추가
plt.annotate('South Korea : ' + str(korea_rank + 1),
             xy = (korea_rank, korea_alcohol_rate),
             xytext = (korea_rank + 10, korea_alcohol_rate + 1.5),
             arrowprops= dict(facecolor = 'r', shrink = 0.05))

plt.show()